In [1]:
# %% [markdown]
# # Pre-Registered Universal Suite (δ, φ, γ) — Bias-Controlled Tests
#
# **Scope (fixed before running):**
#
# **Expression class** (LHS vs RHS):
# - LHS:  L(a,b) = a * (δ/(δ-1)) + b * √φ   with a,b ∈ {0,1}  (and c = 0)
# - RHS:  R(s)   = exp(γ * s)               with s ∈ {1, φ}
#
# We evaluate **all combinations** of (a,b,s).
# The **target identity** you noticed is the specific combo (a,b,s) = (1,1,φ).
#
# **Objectives implemented:**
# 1) Pre-registered micro-study of the above class; compute errors for all combos;
#    run a Monte Carlo null (constants replaced by random irrationals of similar scale)
#    to estimate how surprising the best-fit residual is; apply Bonferroni implicitly
#    by comparing **min residual** across the full grid to the null.
# 2) Universality sweep: numerically estimate Feigenbaum δ_z (no internet/lookups)
#    using **superstable** parameters for f_z(x) = 1 - μ |x|^z on [-1,1].
#    Then test   δ_z/(δ_z - 1) + √φ  vs  exp(γ φ)   across z ∈ Z_LIST.
# 3) Golden extremality (constant-space, no dynamics): compute the "best-fit" ŝ = ln(LHS)/γ,
#    check |ŝ - φ|, and compare against metallic means M_m = (m + sqrt(m^2+4))/2 (m=1..M_MAX).
#    Also swap √φ → √M_m and recheck residuals. This avoids bias toward φ.
# 4) Mechanistic lemma scaffold: we print a concise, fixed outline stating *why*
#    the sum-vs-exp structure is being tested (for your appendix).
#
# All outputs are saved to CSVs and PNGs for reproducibility.

# %% [code]
import math, random, csv, json, statistics, sys, time
from dataclasses import dataclass
from typing import List, Tuple, Dict
import numpy as np
import mpmath as mp
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Precision / reproducibility ----------
mp.mp.dps = 120  # high precision for constants and root-finds
RNG = random.Random(1337)  # fixed seed for Monte Carlo null

# ---------- Canonical constants (computed, not looked up) ----------
phi = (1 + mp.sqrt(5)) / 2
sqrt_phi = mp.sqrt(phi)
gamma = mp.euler  # Euler–Mascheroni

# ---------- Unimodal map family and δ_z estimation (superstable roots) ----------

def f_z(x: mp.mpf, mu: mp.mpf, z: mp.mpf) -> mp.mpf:
    # f_z(x) = 1 - mu * |x|^z on [-1,1]
    return 1 - mu * (mp.fabs(x) ** z)

def iterate(x0: mp.mpf, mu: mp.mpf, z: mp.mpf, k: int) -> mp.mpf:
    x = mp.mpf(x0)
    for _ in range(k):
        x = f_z(x, mu, z)
        # keep in bounds (numerical safeguards)
        if x > 1: x = mp.mpf(1)
        if x < -1: x = mp.mpf(-1)
    return x

def F_superstable(mu: mp.mpf, z: mp.mpf, n: int) -> mp.mpf:
    """
    Superstable condition: f^{(2^n)}(0) = 0.
    We solve F_n(mu) = iterate(0, mu, z, 2**n) = 0
    using sign-bracketing + bisection.
    """
    return iterate(mp.mpf('0'), mu, z, 1 << n)

def find_superstable_mu(z: float, n: int,
                        mu_min: float=0.0, mu_max: float=2.0,
                        coarse: int=8000, tol: float=1e-28, max_iter: int=100) -> mp.mpf:
    """
    Find μ_n'(z) where f^{(2^n)}(0) = 0 by:
      1) coarse scan to bracket a sign change
      2) bisection on that bracket
    """
    z = mp.mpf(z)
    mu_min = mp.mpf(mu_min)
    mu_max = mp.mpf(mu_max)

    # coarse scan
    grid = [mu_min + (mu_max - mu_min) * mp.mpf(i) / (coarse - 1) for i in range(coarse)]
    vals = [F_superstable(mu, z, n) for mu in grid]

    bracket = None
    for i in range(len(grid) - 1):
        if vals[i] == 0:
            return grid[i]
        if vals[i] * vals[i+1] < 0:
            bracket = (grid[i], grid[i+1])
            break

    if bracket is None:
        # If no sign change found, try expanding or refining search.
        # We'll do a progressive refinement around the median μ where |F| is smallest.
        idx = int(np.argmin([abs(v) for v in vals]))
        center = grid[idx]
        span = mp.mpf('0.2')
        for _ in range(3):
            a = max(mu_min, center - span)
            b = min(mu_max, center + span)
            fine = 8000
            grid2 = [a + (b - a) * mp.mpf(i) / (fine - 1) for i in range(fine)]
            vals2 = [F_superstable(mu, z, n) for mu in grid2]
            for j in range(len(grid2) - 1):
                if vals2[j] == 0:
                    return grid2[j]
                if vals2[j] * vals2[j+1] < 0:
                    bracket = (grid2[j], grid2[j+1])
                    break
            if bracket is not None:
                break
            # shrink span around new best
            idx = int(np.argmin([abs(v) for v in vals2]))
            center = grid2[idx]
            span *= mp.mpf('0.5')


    if bracket is None:
        raise RuntimeError(f"No sign change for superstable root (z={z}, n={n}). Try larger 'coarse' or different range.")

    a, b = bracket
    fa = F_superstable(a, z, n)
    fb = F_superstable(b, z, n)
    if fa == 0: return a
    if fb == 0: return b

    # bisection
    for _ in range(max_iter):
        m = (a + b) / 2
        fm = F_superstable(m, z, n)
        if fm == 0 or abs(b - a) < tol:
            return m
        if fa * fm < 0:
            b = m; fb = fm
        else:
            a = m; fa = fm
    return (a + b) / 2

def estimate_delta_z(z: float,
                     n_start: int=2, n_end: int=8,
                     coarse: int=8000) -> Tuple[mp.mpf, List[mp.mpf], List[mp.mpf]]:
    """
    Estimate Feigenbaum δ_z using superstable μ_n'(z) roots:
        μ_n' solves f^{(2^n)}(0) = 0
    Then δ_n ≈ (μ_{n-1}' - μ_{n-2}') / (μ_n' - μ_{n-1}')
    Return (δ_est, mu_list, delta_list-by-n)
    """
    mus = []
    for n in range(n_start, n_end + 1):
        try:
            mu_n = find_superstable_mu(z, n, coarse=coarse)
            mus.append(mu_n)
        except RuntimeError as e:
            print(f"Warning: Could not find superstable mu for z={z}, n={n}: {e}")
            # Optionally, you can append a placeholder or skip this n
            # For now, we just print a warning and continue to the next n
            pass


    deltas = []
    for i in range(2, len(mus)):
        num = mus[i-1] - mus[i-2]
        den = mus[i]   - mus[i-1]
        if abs(den) < mp.mpf(1e-50): # Check if denominator is close to zero
            print(f"Warning: Denominator close to zero for z={z}, n={n_start+i}. Skipping delta calculation for this step.")
            continue # Skip this delta calculation
        deltas.append(num / den)
    if not deltas: # If deltas list is empty due to skipping, raise an error or return None
        raise RuntimeError(f"Could not estimate delta for z={z}. Denominators were too close to zero or no superstable mus were found.")
    return deltas[-1], mus, deltas

# ---------- Pre-registered micro-study (expression class) ----------

def lhs_value(delta: mp.mpf, a: int, b: int) -> mp.mpf:
    return (mp.mpf(a) * (delta / (delta - 1))) + (mp.mpf(b) * sqrt_phi)

def rhs_value(s: mp.mpf) -> mp.mpf:
    return mp.e ** (gamma * s)

EXPR_GRID = [(a,b,s) for a in (0,1) for b in (0,1) for s in (mp.mpf(1), phi)]

def evaluate_expression_grid(delta: mp.mpf) -> pd.DataFrame:
    rows = []
    for (a,b,s) in EXPR_GRID:
        L = lhs_value(delta, a, b)
        R = rhs_value(s)
        abs_err = abs(L - R)
        rel_err = abs_err / abs(R)
        rows.append({
            "a": int(a), "b": int(b),
            "s_label": "1" if s == 1 else "phi",
            "LHS": str(L), "RHS": str(R),
            "abs_err": float(abs_err),
            "rel_err": float(rel_err)
        })
    df = pd.DataFrame(rows).sort_values("rel_err").reset_index(drop=True)
    return df

# Monte Carlo null: replace (δ, φ, γ) by random irrationals of similar scale
def random_irrational_like(val: float, span: float, rng: random.Random) -> mp.mpf:
    # Draw around val with ±span fraction, ensuring irrational by adding sqrt(2)*tiny noise
    base = val * (1 + (2*rng.random() - 1) * span)
    tiny = 1e-12 * (2*rng.random() - 1)
    return mp.mpf(base) + mp.sqrt(2) * tiny

def monte_carlo_null(num_trials: int,
                     span_delta: float = 0.25,  # ±25%
                     span_phi: float   = 0.05,  # ±5%
                     span_gamma: float = 0.05) -> List[float]:
    mins = []
    for _ in range(num_trials):
        delta_r = random_irrational_like(4.6692, span_delta, RNG)
        phi_r   = random_irrational_like(float(phi), span_phi, RNG)
        gamma_r = random_irrational_like(float(gamma), span_gamma, RNG)
        sqrt_phi_r = mp.sqrt(phi_r)
        # Evaluate min rel_err over the whole grid for this random triple
        best = mp.inf
        for (a,b,s_choice) in EXPR_GRID:
            L = (mp.mpf(a) * (delta_r / (delta_r - 1))) + (mp.mpf(b) * sqrt_phi_r)
            s_val = 1 if s_choice == 1 else phi_r
            R = mp.e ** (gamma_r * s_val)
            err = abs(L - R) / abs(R)
            if err < best:
                best = err
        mins.append(float(best))
    return mins

# ---------- Golden extremality checks (constant-space) ----------

def metallic_mean(m: int) -> mp.mpf:
    # M_m = (m + sqrt(m^2 + 4)) / 2
    m = mp.mpf(m)
    return (m + mp.sqrt(m*m + 4)) / 2

def golden_extremality_table(delta: mp.mpf, M_max: int = 8) -> pd.DataFrame:
    """
    For the *observed* LHS with sqrt(φ), compute ŝ = ln(LHS)/γ and |ŝ-φ|.
    Then, for m=1..M_max, replace √φ -> √M_m and recompute:
      - residual to exp(γ φ)
      - best-fit ŝ_m
      - |ŝ_m - φ|
    """
    rows = []
    # baseline with true sqrt(φ)
    L_phi = (delta / (delta - 1)) + sqrt_phi
    s_hat = mp.log(L_phi) / gamma
    rows.append({
        "mode": "baseline",
        "m": 1,
        "metallic_mean": str(phi),
        "sqrt_metallic": str(sqrt_phi),
        "LHS": str(L_phi),
        "best_s_hat": float(s_hat),
        "abs_best_s_minus_phi": float(abs(s_hat - phi)),
        "rel_err_vs_exp_gamma_phi": float(abs(L_phi - mp.e**(gamma*phi)) / (mp.e**(gamma*phi)))
    })
    # alternatives
    for m in range(1, M_max + 1):
        M_m = metallic_mean(m)
        L_m = (delta / (delta - 1)) + mp.sqrt(M_m)
        s_hat_m = mp.log(L_m) / gamma
        rows.append({
            "mode": "swap_sqrt_phi",
            "m": m,
            "metallic_mean": str(M_m),
            "sqrt_metallic": str(mp.sqrt(M_m)),
            "LHS": str(L_m),
            "best_s_hat": float(s_hat_m),
            "abs_best_s_minus_phi": float(abs(s_hat_m - phi)),
            "rel_err_vs_exp_gamma_phi": float(abs(L_m - mp.e**(gamma*phi)) / (mp.e**(gamma*phi)))
        })
    return pd.DataFrame(rows)

# ---------- Run the suite ----------

OUT_PREFIX = "delta_phi_gamma_suite"
Z_LIST = [1.5, 2.0, 2.5, 3.0]   # can extend later
N_START, N_END = 2, 8           # superstable orders (2^n), increase for more accuracy
COARSE_SCAN = 8000              # coarse scan granularity (increase for more stability)
NULL_TRIALS = 4000              # Monte Carlo size (increase as needed)

print("== Mechanistic scaffold (fixed text) ==")
print("""
We test a scaling balance between (i) dyadic renormalization gain and (ii) a golden
mid-inflation step against (iii) a discrete–continuous correction budget:
    LHS = δ/(δ−1) + √φ   versus   RHS = exp(γ · s)
Geometric reading: the cascade’s amplification plus a quasi-periodic half-inflation
is matched by an exponentiated Euler–Mascheroni correction, evaluated at scale s.
The pre-registered class fixes LHS to {0, δ/(δ−1)} + {0, √φ} and s ∈ {1, φ}.
All combinations are evaluated; selection bias is controlled by a Monte Carlo null
that replaces (δ, φ, γ) with random irrationals of similar magnitude.
""")

# (2) Estimate δ for z=2 (baseline), then do universality sweep
delta_estimates = []
for z in Z_LIST:
    t0 = time.time()
    try:
        dz, mus, dlist = estimate_delta_z(z, n_start=N_START, n_end=N_END, coarse=COARSE_SCAN)
        dt = time.time() - t0
        delta_estimates.append({"z": z,
                                "delta_est": float(dz),
                                "mus": [str(m) for m in mus],
                                "delta_by_n": [str(dv) for dv in dlist],
                                "seconds": dt})
        print(f"z={z}: δ_est ≈ {dz}  (computed in {dt:.1f}s)")
    except RuntimeError as e:
        print(f"Could not estimate delta for z={z}: {e}")


df_delta = pd.DataFrame(delta_estimates)

if df_delta.empty:
    print("\nDelta estimation failed for all z values. Cannot proceed with calculations requiring delta.")
else:
    df_delta.to_csv(f"{OUT_PREFIX}_delta_sweep.csv", index=False)
    print("\nSaved:", f"{OUT_PREFIX}_delta_sweep.csv")

    # Pick δ for the micro-study from z=2.0 (closest to quadratic universality class)
    try:
        delta_quadratic = mp.mpf(df_delta[df_delta['z']==2.0]['delta_est'].iloc[0])
    except Exception:
        # fallback to the best-available δ
        delta_quadratic = mp.mpf(df_delta.iloc[0]['delta_est'])

    print("\nBaseline δ (z=2.0) used in expression grid:", delta_quadratic)

    # (1) Pre-registered expression grid
    df_grid = evaluate_expression_grid(delta_quadratic)
    df_grid.to_csv(f"{OUT_PREFIX}_expression_grid.csv", index=False)
    print("\nExpression grid (sorted by rel_err):")
    display(df_grid)
    print("Saved:", f"{OUT_PREFIX}_expression_grid.csv")

    best_rel_err = float(df_grid['rel_err'].min())
    best_row = df_grid.iloc[0].to_dict()
    print("\nBest combo:", json.dumps(best_row, indent=2))

    # Monte Carlo null
    print(f"\nRunning Monte Carlo null with {NULL_TRIALS} trials ...")
    null_mins = monte_carlo_null(NULL_TRIALS)
    df_null = pd.DataFrame({"min_rel_err": null_mins})
    df_null.to_csv(f"{OUT_PREFIX}_null_montecarlo.csv", index=False)
    p_hat = sum(1 for x in null_mins if x <= best_rel_err) / len(null_mins)
    print(f"Observed best rel_err = {best_rel_err:.8e}")
    print(f"Monte Carlo p-hat (min-over-grid) = {p_hat:.6f}")
    print("Saved:", f"{OUT_PREFIX}_null_montecarlo.csv")

    plt.figure(figsize=(6,4))
    plt.hist(null_mins, bins=40)
    plt.axvline(best_rel_err, linestyle='--', linewidth=2)
    plt.xlabel("Min relative error over expression grid (null trials)")
    plt.ylabel("Count")
    plt.title("Monte Carlo Null — Selection-Controlled")
    plt.tight_layout()
    plt.savefig(f"{OUT_PREFIX}_null_hist.png", dpi=160)
    plt.show()
    print("Saved plot:", f"{OUT_PREFIX}_null_hist.png")

    # (2) Universality check table: residuals with δ_z
    rows_uni = []
    target_R = mp.e ** (gamma * phi)
    for rec in delta_estimates:
        dz = mp.mpf(rec["delta_est"])
        L = (dz / (dz - 1)) + sqrt_phi
        abs_err = abs(L - target_R)
        rel_err = abs_err / abs(target_R)
        rows_uni.append({
            "z": rec["z"],
            "delta_est": rec["delta_est"],
            "lhs": str(L),
            "rhs_exp_gamma_phi": str(target_R),
            "abs_err": float(abs_err),
            "rel_err": float(rel_err)
        })
    df_uni = pd.DataFrame(rows_uni).sort_values("rel_err").reset_index(drop=True)
    display(df_uni)
    df_uni.to_csv(f"{OUT_PREFIX}_universality_residuals.csv", index=False)
    print("Saved:", f"{OUT_PREFIX}_universality_residuals.csv")

    # (3) Golden extremality (constant-space)
    df_golden = golden_extremality_table(delta_quadratic, M_max=10)
    display(df_golden.sort_values(["mode","abs_best_s_minus_phi"]).reset_index(drop=True))
    df_golden.to_csv(f"{OUT_PREFIX}_golden_extremality.csv", index=False)
    print("Saved:", f"{OUT_PREFIX}_golden_extremality.csv")

    # Quick plot: |s_hat - φ| for metallic means swap
    df_swap = df_golden[df_golden["mode"]=="swap_sqrt_phi"].copy()
    plt.figure(figsize=(6,4))
    plt.plot(df_swap["m"], df_swap["abs_best_s_minus_phi"], marker='o')
    plt.xlabel("m (metallic mean index)")
    plt.ylabel("| best s_hat  −  φ |")
    plt.title("Golden Extremality (constant-space) — proximity of ŝ to φ")
    plt.tight_layout()
    plt.savefig(f"{OUT_PREFIX}_golden_extremality_proximity.png", dpi=160)
    plt.show()
    print("Saved plot:", f"{OUT_PREFIX}_golden_extremality_proximity.png")

    # (4) Print a static lemma outline (for your appendix)
    print("\n== Appendix: Mechanistic lemma (outline, fixed text) ==")
    print(r"""
(α) Dyadic RG near the period-doubling accumulation point linearizes to a scale gain
    of the form δ/(δ−1) in any additive “scale-budget” coordinate (parameter-space zoom).

(β) Quasi-periodic inflation/deflation in the golden module admits a natural half-step
    √φ as the mid-scale between inflation levels (e.g., in Penrose/Beatty constructions).

(γ) Discrete–continuous comparisons produce a universal Euler–Mascheroni offset γ
    (sum vs log), which exponentiates to exp(γ·s) when cast as a multiplicative capacity
    at an incommensurate scale s. Choosing s=φ encodes maximal incommensurability.

(δ) Therefore a boundary law of the form:  δ/(δ−1) + √φ ≈ exp(γ·φ)
    is a natural balance candidate at the quasi-periodic ↔ period-doubling interface.

This outline motivates the tests above but does not assume their truth.
""")

    print("\n== Done. CSVs/PNGs are saved with prefix:", OUT_PREFIX, "==")

== Mechanistic scaffold (fixed text) ==

We test a scaling balance between (i) dyadic renormalization gain and (ii) a golden
mid-inflation step against (iii) a discrete–continuous correction budget:
    LHS = δ/(δ−1) + √φ   versus   RHS = exp(γ · s)
Geometric reading: the cascade’s amplification plus a quasi-periodic half-inflation
is matched by an exponentiated Euler–Mascheroni correction, evaluated at scale s.
The pre-registered class fixes LHS to {0, δ/(δ−1)} + {0, √φ} and s ∈ {1, φ}.
All combinations are evaluated; selection bias is controlled by a Monte Carlo null
that replaces (δ, φ, γ) with random irrationals of similar magnitude.

Could not estimate delta for z=1.5: Could not estimate delta for z=1.5. Denominators were too close to zero or no superstable mus were found.
Could not estimate delta for z=2.0: Could not estimate delta for z=2.0. Denominators were too close to zero or no superstable mus were found.
Could not estimate delta for z=2.5: Could not estimate delta for z=2.